# 이상치 처리 작전

In [39]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

In [81]:
df = pd.read_csv('train.csv').drop('ID', axis=1)

## 1. Z-score

In [78]:
# cols = ['공복 혈당', '혈압', '중성 지방', '혈청 크레아티닌', '콜레스테롤', '고밀도지단백', '저밀도지단백', '헤모글로빈', '간 효소율']

# fig, axs = plt.subplots(3, 3, figsize=(15, 10))

# for i, col in enumerate(cols):
#     row = i // 3
#     col_idx = i % 3
#     ax = axs[row, col_idx]

#     data = df[col]
#     z_scores = np.abs(stats.zscore(data))
#     threshold = 3

#     outlier_indices = np.where(z_scores > threshold)[0]
#     outliers = data[outlier_indices]

#     ax.scatter(range(len(data)), data, c='blue', alpha=0.2)
#     ax.scatter(outlier_indices, outliers, c='red', label='Outliers', alpha=0.5)
#     ax.axhline(y=np.mean(data) + 3*np.std(data), color='r', linestyle='--', label='3σ threshold')
#     ax.axhline(y=np.mean(data) - 3*np.std(data), color='r', linestyle='--')
#     ax.legend()

# plt.tight_layout()
# plt.show()

In [105]:
cols = ['공복 혈당', '혈압', '중성 지방', '혈청 크레아티닌', '콜레스테롤', '고밀도지단백', '저밀도지단백', '헤모글로빈', '간 효소율']
z_scores = np.abs(stats.zscore(X))
threshold = 3
outliers = np.where(z_scores > threshold)

for col in cols:
    df['z_score'] = stats.zscore(df[col])
    outlier_zscore = df[(df['z_score'].abs() > 3)]

    print(f'[{col}]')
    print(f'이상치 개수 : {len(outlier_zscore)}개 ({len(outlier_zscore)/7000:.2%})')
    print('='*25)

[공복 혈당]
이상치 개수 : 124개 (1.77%)
[혈압]
이상치 개수 : 49개 (0.70%)
[중성 지방]
이상치 개수 : 112개 (1.60%)
[혈청 크레아티닌]
이상치 개수 : 13개 (0.19%)
[콜레스테롤]
이상치 개수 : 36개 (0.51%)
[고밀도지단백]
이상치 개수 : 62개 (0.89%)
[저밀도지단백]
이상치 개수 : 15개 (0.21%)
[헤모글로빈]
이상치 개수 : 61개 (0.87%)
[간 효소율]
이상치 개수 : 64개 (0.91%)


In [95]:
cols = ['공복 혈당', '혈압', '중성 지방', '혈청 크레아티닌', '콜레스테롤', '고밀도지단백', '저밀도지단백', '헤모글로빈', '간 효소율']
for col in cols:
    Q1 = np.percentile(df[col], 25)
    Q3 = np.percentile(df[col], 75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers_mask = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f'[{col}] Q1 : {Q1:.2f}, Q3 : {Q3:.2f}, IQR : {IQR:.2f} lower : {lower_bound:.2f}, upper : {upper_bound:.2f}')
    print(f'이상치 개수 : {len(outliers_mask)}개 ({(len(outliers_mask))/7000:.2%})')
    print('='*80)

[공복 혈당] Q1 : 89.00, Q3 : 104.00, IQR : 15.00 lower : 66.50, upper : 126.50
이상치 개수 : 402개 (5.74%)
[혈압] Q1 : 40.00, Q3 : 50.00, IQR : 10.00 lower : 25.00, upper : 65.00
이상치 개수 : 176개 (2.51%)
[중성 지방] Q1 : 74.00, Q3 : 161.00, IQR : 87.00 lower : -56.50, upper : 291.50
이상치 개수 : 296개 (4.23%)
[혈청 크레아티닌] Q1 : 0.80, Q3 : 1.00, IQR : 0.20 lower : 0.50, upper : 1.30
이상치 개수 : 395개 (5.64%)
[콜레스테롤] Q1 : 173.00, Q3 : 219.00, IQR : 46.00 lower : 104.00, upper : 288.00
이상치 개수 : 113개 (1.61%)
[고밀도지단백] Q1 : 47.00, Q3 : 66.00, IQR : 19.00 lower : 18.50, upper : 94.50
이상치 개수 : 124개 (1.77%)
[저밀도지단백] Q1 : 92.00, Q3 : 136.00, IQR : 44.00 lower : 26.00, upper : 202.00
이상치 개수 : 85개 (1.21%)
[헤모글로빈] Q1 : 13.60, Q3 : 15.70, IQR : 2.10 lower : 10.45, upper : 18.85
이상치 개수 : 87개 (1.24%)
[간 효소율] Q1 : 0.84, Q3 : 1.38, IQR : 0.54 lower : 0.03, upper : 2.19
이상치 개수 : 135개 (1.93%)


## 3. 도메인 지식

In [103]:
# [시력] 소숫점 정리, 측정불가 2.0 이상 -> 최저값 0.1로 변경
o1 = len(df.loc[df['시력'] > 2.0])
print(f'[시력] 이상치 개수 : {o1}개 ({o1/7000:.2%})')
print('='*40)

# [중성 지방] 500 초과, 상한선 500으로 처리
o2 = len(df.loc[df['중성 지방'] > 500])
print(f'[중성 지방] 이상치 개수 : {o2}개 ({o2/7000:.2%})')
print('='*40)
    
# # [혈청 크레아티닌] 3 초과, 상한선 3으로 처리
o3 = len(df.loc[df['혈청 크레아티닌'] > 3.0])
print(f'[혈청 크레아티닌] 이상치 개수 : {o3}개 ({o3/7000:.2%})')
print('='*40)
    
# [간 효소율] 5 초과, 상한선 5.0으로 처리
o4 = len(df.loc[df['간 효소율'] > 5.0])
print(f'[간 효소율] 이상치 개수 : {o4}개 ({o4/7000:.2%})')
print('='*40)

# [저밀도지단백] 중성지방이 400미만이면서 저밀도지단백이 500 초과, 프리데발트 공식으로 처리
o5 = len(df.loc[(df['저밀도지단백'] > 500) & (df['중성 지방'] < 400)])
print(f'[저밀도지단백] 이상치 개수 : {o5}개 ({o5/7000:.2%})')
print('='*40)

[시력] 이상치 개수 : 30개 (0.43%)
[중성 지방] 이상치 개수 : 1개 (0.01%)
[혈청 크레아티닌] 이상치 개수 : 3개 (0.04%)
[간 효소율] 이상치 개수 : 4개 (0.06%)
[저밀도지단백] 이상치 개수 : 5개 (0.07%)
